In [ ]:
import os
os.environ["HF_HUB_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/hub"
os.environ["HF_DATASETS_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/datasets"

In [ ]:
# from datasets import load_dataset
# ds = load_dataset("nyu-mll/multi_nli")

import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation_matched': 'data/validation_matched-00000-of-00001.parquet', 'validation_mismatched': 'data/validation_mismatched-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["train"])
df_val_matched = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["validation_matched"])
df_val_mismatched = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["validation_mismatched"])

In [ ]:
label_map = {
    1: "neutral",
    0: "entailment",
    2: "contradiction",
}

In [ ]:
import random
def process_columns(df):
    random.seed(5)
    df['label_flipped'] = df['label'].apply(
        lambda x: random.choice([label for label in [0, 1, 2] if label != x])
    )
    assert (df['label_flipped'] != df['label']).all()
    df['label_str'] = df['label'].map(label_map)
    df['label_flipped_str'] = df['label_flipped'].map(label_map)
    df['in-language'] = 'Premise: ' + df["premise"] + '\nHyopthesis: ' + df["hypothesis"] + '\nLabel: ' + df["label_str"]
    df['out-language'] = 'Premise: ' + df["premise"] + '\nHyopthesis: ' + df["hypothesis"] + '\nLabel: ' + df["label_flipped_str"]
    return df

df_train = process_columns(df_train)
df_val_matched = process_columns(df_val_matched)
df_val_mismatched = process_columns(df_val_mismatched)

In [ ]:
instruction = """Provide a classification label for the pair, indicating the relationship between the premise and hypothesis:
- entailment : The hypothesis logically follows from the premise.
- neutral : The hypothesis is neither entailed nor contradicted by the premise.
- contradiction : The hypothesis contradicts the premise.


"""

In [ ]:
print(df_train.iloc[11]['in-language'])

In [ ]:
print(df_train.iloc[11]['out-language'])

In [ ]:
print(df_val_mismatched.iloc[2]['in-language'])

In [ ]:
df_train['genre'].unique()

In [ ]:
df_val_matched.groupby('genre').count()

In [ ]:
df_val_matched['genre'].unique()

In [ ]:
# select one genre only
df_train = df_train[df_train['genre'] == 'government']
df_val_matched = df_val_matched[df_val_matched['genre'] == 'government']

df_train.shape, df_val_matched.shape, df_val_mismatched.shape

In [ ]:
df_val_mismatched['genre'].unique()

In [ ]:
df_train['in-language_len'] = df_train['in-language'].apply(lambda x: len(x.strip()))
df_train['out-language_len'] = df_train['out-language'].apply(lambda x: len(x.strip()))
df_val_matched['in-language_len'] = df_val_matched['in-language'].apply(lambda x: len(x.strip()))
df_val_matched['out-language_len'] = df_val_matched['out-language'].apply(lambda x: len(x.strip()))
df_val_mismatched['in-language_len'] = df_val_mismatched['in-language'].apply(lambda x: len(x.strip()))
df_val_mismatched['out-language_len'] = df_val_mismatched['out-language'].apply(lambda x: len(x.strip()))

In [ ]:
df_train[df_train['in-language_len'] == 0].shape, df_train[df_train['out-language_len'] == 0].shape, df_val_matched[df_val_matched['in-language_len'] == 0].shape, df_val_matched[df_val_matched['out-language_len'] == 0].shape, df_val_mismatched[df_val_mismatched['in-language_len'] == 0].shape, df_val_mismatched[df_val_mismatched['out-language_len'] == 0].shape

In [ ]:
fa

In [ ]:
mnli_dataset_in_distribution_dict = {
    "train_sequences": df_train['in-language'].tolist(),
    "test_sequences": df_val_matched['in-language'].tolist(),
    "non_grammatical_test_sequences_edit_distance_1": df_val_matched['out-language'].tolist(),
}


mnli_dataset_out_distribution_dict = {
    "train_sequences": df_train['in-language'].tolist(),
    "test_sequences": df_val_mismatched['in-language'].tolist(),
    "non_grammatical_test_sequences_edit_distance_1": df_val_mismatched['out-language'].tolist(),
}

In [ ]:
print(mnli_dataset_in_distribution_dict["test_sequences"][0])

In [ ]:
print(mnli_dataset_in_distribution_dict["non_grammatical_test_sequences_edit_distance_1"][0])

In [ ]:
print(mnli_dataset_out_distribution_dict["test_sequences"][0])

In [ ]:
print(mnli_dataset_out_distribution_dict["non_grammatical_test_sequences_edit_distance_1"][0])

In [ ]:
filename_in_distribution = "../data/mnli_dataset_in_distribution/sequences_w_edit_distance_mnli_dataset_in_distribution_10000_5.pkl"
filename_out_distribution = "../data/mnli_dataset_out_distribution/sequences_w_edit_distance_mnli_dataset_out_distribution_10000_5.pkl"

os.system(f"mkdir -p ../data/mnli_dataset_in_distribution")
os.system(f"mkdir -p ../data/mnli_dataset_out_distribution")


filename_in_distribution_instruction = "../data/mnli_dataset_in_distribution/instruction_mnli_dataset_in_distribution.pkl"
filename_out_distribution_instruction = "../data/mnli_dataset_out_distribution/instruction_mnli_dataset_out_distribution.pkl"


import pickle

with open(filename_in_distribution, 'wb') as f:
    pickle.dump(mnli_dataset_in_distribution_dict, f)

with open(filename_out_distribution, 'wb') as f:
    pickle.dump(mnli_dataset_out_distribution_dict, f)

with open(filename_in_distribution_instruction, 'wb') as f:
    pickle.dump(instruction, f)

with open(filename_out_distribution_instruction, 'wb') as f:
    pickle.dump(instruction, f)


In [ ]:
fa

In [ ]:
import os
os.environ["HF_HUB_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/hub"
os.environ["HF_DATASETS_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/datasets"
from transformers import set_seed, AutoTokenizer, AutoModelForCausalLM

In [ ]:
seed = 5
set_seed(seed)
model = "EleutherAI/pythia-1b"
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
tokenizer.decode([    0,  1394,   588,   320,  1677,  6430,   432,   247,  7473,  3448,
           13,  9070,   407,   686,    28,  5983,  5402,  4836,   310,   281,
         6635,   247,   747,  3425,   407,  4715, 43548,  9994,  6127,   432,
          253,  1677,  6430,    15,   209,    23,    21,    22,    26,    25,
           24,    18,    19,    20,    18,    19,    20,    23,    22,    25,
           24,    26,    25,    24,    20,    18,    23,    22,    23,    22,
           18,    19,    20,    26,    25,    24,    20,    18,    25,    24,
           23,    21,    22,    20,    18,    26,    25,    24,    23,    22,
           23,    21,    22,    20,    18,    26,    25,    24,    23,    22,
           25,    24,    18,    19,    20,    28,    23,    21,    22,    26,
           25,    24,    18,    19,    20,    23,    22,    20,    18,    26,
           25,    24,    18,    19,    20,    25,    24,    23,    21,    22,
           23,    21,    22,    26,    25,    24,    18,    19,    20,    23,
           21,    22,    18,    19,    20,    26,    25,    24,    28,    23,
           22,    18,    19,    20,    26,    25,    24,    26,    25,    24,
           20,    18,    23,    21,    22,    23,    22,    20,    18,    25,
           24,    23,    22,    25,    24,    18,    19,    20,    26,    25,
           24,    23,    21,    22,    18,    19,    20,    20,    18,    23,
           22,    26,    25,    24,    26,    25,    24,    23,    22,    18,
           19,    20,    25,    24,    18,    19,    20,    23,    22,    28,
           25,    24,    20,    18,    23,    21,    22,    20,    18,    23,
           22,    25,    24,    18,    19,    20,    23,    21,    22,    25,
           24,    23,    22,    25,    24,    18,    19,    20,    25,    24,
           23,    21,    22,    20,    18,    18,    19,    20,    23,    21,
           22,    25,    24,    18,    19,    20,    26,    25,    24,    23,
           22,    23,    21,    22,    25,    24,    20,    18,    28,    23,
           21,    22,    20,    18,    26,    25,    24,    23,    21,    22,
           26,    25,    24,    18,    19,    20,    23,    22,    26,    25,
           24,    18,    19,    20,    23,    21,    22,    18,    19,    20,
           25,    24,    20,    18,    25,    24,    23,    21,    22,    25,
           24,    23,    22,    20,    18,    18,    19,    20,    23,    22,
           25,    24,    23,    21,    22,    25,    24,    18,    19,    20,
           28,    25,    24,    18,    19,    20,    23,    21,    22,    23,
           22,    20,    18,    26,    25,    24,    20,    18,    25,    24,
           23,    21,    22,    23,    22,    25,    24,    20,    18,    23,
           21,    22,    18,    19,    20,    25,    24,    28,    23,    21,
           22,    25,    24,    18,    19,    20,    20,    18,    23,    21,
           22,    26,    25,    24,    18,    19,    20,    23,    22,    26,
           25,    24,    23,    22,    25,    24,    20,    18,    18,    19,
           20,    26,    25,    24,    23,    21,    22,    25,    24,    23,
           21,    22,    20,    18,    23,    21,    22,    18,    19,    20,
           26,    25,    24,    23,    21,    22,    26,    25,    24,    18,
           19,    20,    28,    26,    25,    24,    18,    19,    20,    23,
           21,    22,    23,    22,    20,    18,    25,    24,    23,    21,
           22,    20,    18,    25,    24,    25,    24,    18,    19,    20,
           23,    21,    22,    18,    19,    20,    26,    25,    24,    23,
           21,    22,    20,    18,    23,    22,    26,    25,    24,    25,
           24,    23,    21,    22,    20,    18,    23,    22,    26,    25,
           24,    20,    18,    28,    25,    24,    18,    19,    20,    23,
           22,    18,    19,    20,    23,    21,    22,    26,    25,    24,
           18,    19,    20,    26,    25,    24,    23,    21,    22,    23,
           21,    22,    18,    19,    20,    26,    25,    24,    23,    21,
           22,    26,    25,    24,    20,    18,    28,    20,    18,    23,
           21,    22,    26,    25,    24,    26,    25,    24,    18,    19,
           20,    23,    21,    22,    20,    18,    25,    24,    23,    21,
           22,    26,    25,    24,    20,    18,    23,    22,    23,    21,
           22,    20,    18,    26,    25,    24,    28,    23,    22,    20,
           18,    26,    25,    24,    25,    24,    18,    19,    20,    23,
           22,    26,    25,    24,    20,    18,    23,    21,    22,    20,
           18,    23,    21,    22,    26,    25,    24,    25,    24,    23,
           21,    22,    20,    18,    26,    25,    24,    23,    22,    20,
           18,    18,    19,    20,    23,    21,    22,    25,    24,    23,
           21,    22,    26,    25,    24,    18,    19,    20,    28,    25,
           24,    18,    19,    20,    23,    21,    22,    23,    22,    18,
           19,    20,    26,    25,    24,    26,    25,    24,    18,    19,
           20,    23,    21,    22,    20,    18,    23,    21,    22,    26,
           25,    24,    20,    18,    26,    25,    24,    23,    21,    22,
           25,    24,    23,    21,    22,    20,    18,    23,    21,    22,
           18,    19,    20,    25,    24,    23,    22,    26,    25,    24,
           20,    18,    28,    18,    19,    20,    23,    22,    26,    25,
           24,    23,    22,    25,    24,    20,    18,    25,    24,    23,
           21,    22,    20,    18,    25,    24,    18,    19,    20,    23,
           21,    22,    23,    21,    22,    18,    19,    20,    25,    24,
           28,    23,    22,    20,    18,    26,    25,    24,    23,    22,
           26,    25,    24,    18,    19,    20,    18,    19,    20,    25,
           24,    23,    22,    23,    21,    22,    18,    19,    20,    25,
           24,    25,    24,    18,    19,    20,    23,    21,    22,    28,
           23,    21,    22,    18,    19,    20,    25,    24,    25,    24,
           18,    19,    20,    23,    21,    22,    20,    18,    26,    25,
           24,    23,    21,    22,    25,    24,    20,    18,    23,    21,
           22,    18,    19,    20,    23,    22,    25,    24,    28,    23,
           22,    18,    19,    20,    26,    25,    24,    25,    24,    20,
           18,    23,    22,    20,    18,    26,    25,    24,    23,    21,
           22,    20,    18,    23,    22,    25,    24,    23,    22,    25,
           24,    20,    18,    28,     0])

In [ ]:
tokenizer.decode([49,  83,  70,  78,  74,  84,  70,  27, 209,   8,  42,
        209,  84,  66,  88, 209,  73,  74,  78, 209,  72,  70,  85, 209,  66,
         67,  80,  66,  83,  69, 209,  78,  90,  84,  70,  77,  71,  15, 187,
         41,  90,  80,  81,  85,  73,  70,  84,  74,  84,  27, 209,  42, 209,
         84,  66,  88, 209,  73,  74,  78, 209,  72,  70,  85, 209,  80,  79,
        209,  85,  73,  70, 209,  85,  83,  66,  74,  79,  15, 187,  45,  66,
         67,  70,  77,  27, 209,  68,  80,  79,  85,  83,  66,  69,  74,  68,
         85,  74,  80,  79,   0])

In [ ]:
tokenizer.encode("Premise: I saw him get aboard myself.\nHyopthesis: I saw him get on the train.\nLabel: contradiction")

In [ ]:
tokenizer.decode([    0,   831,   535,   533,   535,  4129,   535, 22955,   535,  1723,
          535,     0])